In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import json
from src.pokemon import Pokemon, PokemonFactory, StatusEffect
from src.catching import attempt_catch

In [ ]:
available_pokemon = []
with open("pokemon.json", "r") as file:
    pokemon_db = json.load(file)
    available_pokemon = [key for key in pokemon_db]

available_pokemon

In [ ]:
def run_iterations(pokemon_name, pokeball, level, status, hp_percentage, iterations):
    factory = PokemonFactory("pokemon.json")
    pokemon = factory.create(pokemon_name, level, status, hp_percentage)
    success = 0 
    for _ in range(iterations):
        attempt, rate = attempt_catch(pokemon, pokeball)
        if attempt:
            success += 1
    return [pokemon_name, pokeball, success, iterations]

In [ ]:
ITERS = 100
LEVEL = 100
STATUS = StatusEffect.NONE
HP_PERCENTAGE = 1.0
POKEBALLS = ["pokeball", "ultraball", "fastball", "heavyball"]

results = []
df = pd.DataFrame(columns= ["pokemon_name", "pokeball", "success", "attemps"])
for pokeball in POKEBALLS:
    for pokemon_name in available_pokemon:
        new_row = run_iterations(pokemon_name, pokeball, LEVEL, STATUS, HP_PERCENTAGE, ITERS)
        df.loc[len(df)] = new_row

df['accuracy'] = df.success / df.attemps

df = df.groupby("pokemon_name").agg({"accuracy": ["mean", "std"]}).reset_index()
df.columns = df.columns.map("_".join)
df

In [ ]:
# Crear una figura y un eje
fig, ax = plt.subplots(figsize=(10, 6))

# Crear las barras del gráfico
x = df["pokemon_name_"]
y = df["accuracy_mean"]
error = df["accuracy_std"]
ax.bar(x, y, yerr=error, capsize=5)

# Configurar el eje x
ax.set_xticks(range(len(x)))
ax.set_xticklabels(x, rotation=45, ha="right")
ax.set_xlabel("Pokemon")

# Configurar el eje y
ax.set_ylabel("Accuracy")

# Añadir título
ax.set_title("Promedio de capturas de los distintos Pokemons en 100 iteraciones")

# Mostrar el gráfico
plt.show()